In [1]:
import h5py
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

/BRAIN/ckolling-phd/work/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
code_path = "/BRAIN/neuroai_project/work/cajal_llm_project"
code_path

'/BRAIN/neuroai_project/work/cajal_llm_project'

In [3]:
import sys
sys.path.append(code_path)

from encoding import *
from preprocessing import get_ordered_representations, normalize_train_test, concat_past_features, lanczosinterp2D, delete_block_edges
from encoding import nested_blocked_cv, ridge_regression_fit_sklearn, ridge_regression_predict_torch
from analysis import pearson_correlation

All tests passed for pearson_correlation function.


## 0. load fmri data and annotation

In [4]:
data_path_prefix = os.path.join(code_path, "data/HP_data/fMRI")
HF_home = "/SWS/llms/nobackup/"
results_path_prefix = os.path.join(code_path, "results")

In [5]:
# raw fmri data for one subject
fmri_data = np.load(f"{data_path_prefix}/data_subject_I.npy", allow_pickle=True)
fmri_data.shape, fmri_data[:10]

((1211, 25263),
 array([[-1.39391795e+00, -4.95610000e-01, -1.76625698e+00, ...,
         -7.66158058e-01, -2.32445619e-01, -1.46871563e-01],
        [ 5.30454149e-01, -2.32485978e-01,  3.52580458e-01, ...,
         -2.31855518e-03, -2.46482123e-01, -4.29284279e-01],
        [ 1.13308842e+00,  3.31803866e-01, -3.14589030e-01, ...,
          8.15993870e-01,  1.31184511e+00,  1.38407545e+00],
        ...,
        [-1.10624234e+00, -1.08674185e+00, -5.44766153e-01, ...,
          6.70158027e-01,  9.81612876e-01,  8.54572439e-01],
        [-1.58241086e+00, -1.14630569e+00, -5.57836731e-01, ...,
         -3.30246991e-01, -4.22094739e-01, -1.26951335e-01],
        [-2.59743061e+00, -1.55403741e+00, -1.93496114e+00, ...,
          1.01716606e+00,  2.10392591e-01,  1.11087041e-01]],
       shape=(10, 25263)))

In [6]:
# timing of each fmri TRs in seconds
fmri_time = np.load(f"{data_path_prefix}/time_fmri.npy", allow_pickle=True)
fmri_time.shape, fmri_time[:10]

((1351,), array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18], dtype=uint16))

In [7]:
# indices of which TRs belong to which run.
fmri_runs = np.load(f"{data_path_prefix}/runs_fmri.npy", allow_pickle=True)
fmri_runs.shape, fmri_runs[:10]

((1351,), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint16))

In [8]:
# list of words shown as stimuli (sequentially on a screen word by word)
stimuli_words = np.load(f"{data_path_prefix}/words_fmri.npy", allow_pickle=True)
stimuli_words.shape, stimuli_words[:10]

((5176,),
 array(['Harry', 'had', 'never', 'believed', 'he', 'would', 'meet', 'a',
        'boy', 'he'], dtype='<U14'))

In [9]:
len(" ".join(stimuli_words).split("+"))

202

In [10]:
# timing in seconds of the words
word_times = np.load(f"{data_path_prefix}/time_words_fmri.npy", allow_pickle=True)
word_times.shape, word_times[:10]

((5176,), array([20. , 20.5, 21. , 21.5, 22. , 22.5, 23. , 23.5, 24. , 24.5]))

## 1. prepare the input sequence for the LLM
(e.g. by changing formatting). Here we change "+" to "\n\n" and "@" to nothing (used to highlight italics)

In [11]:
LLM_input_sequence = []
for word in stimuli_words:
    LLM_input_sequence.append(word) if word != "+" and not "@" in word \
    else LLM_input_sequence.append(word.replace("+", "\n\n").replace("@",""))
assert len(word_times.tolist()) == len(LLM_input_sequence), "different length" # make sure we still have the same length as we have word timings.

### Shuffling words

In [12]:
LLM_input_sequence

[np.str_('Harry'),
 np.str_('had'),
 np.str_('never'),
 np.str_('believed'),
 np.str_('he'),
 np.str_('would'),
 np.str_('meet'),
 np.str_('a'),
 np.str_('boy'),
 np.str_('he'),
 np.str_('hated'),
 np.str_('more'),
 np.str_('than'),
 np.str_('Dudley,'),
 np.str_('but'),
 np.str_('that'),
 np.str_('was'),
 np.str_('before'),
 np.str_('he'),
 np.str_('met'),
 np.str_('Draco'),
 np.str_('Malfoy.'),
 np.str_('Still,'),
 np.str_('first-year'),
 np.str_('Gryffindors'),
 np.str_('only'),
 np.str_('had'),
 np.str_('Potions'),
 np.str_('with'),
 np.str_('the'),
 np.str_('Slytherins,'),
 np.str_('so'),
 np.str_('they'),
 np.str_("didn't"),
 np.str_('have'),
 np.str_('to'),
 np.str_('put'),
 np.str_('up'),
 np.str_('with'),
 np.str_('Malfoy'),
 np.str_('much.'),
 np.str_('Or'),
 np.str_('at'),
 np.str_('least,'),
 np.str_('they'),
 np.str_("didn't"),
 np.str_('until'),
 np.str_('they'),
 np.str_('spotted'),
 np.str_('a'),
 np.str_('notice'),
 np.str_('pinned'),
 np.str_('up'),
 np.str_('in'),
 np

## 2. compute LLM representations for these words.

In [13]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir=HF_home)
model = AutoModel.from_pretrained(model_name, cache_dir=HF_home).to("cuda")
print("Loaded model and tokenizer.")

Loaded model and tokenizer.


In [14]:
strings_to_find = LLM_input_sequence # we search for these words in sequential order in the full prompt to obtain avg representations for these.
prompt_text = " ".join(LLM_input_sequence)

In [15]:
prompt_text[:500], len(prompt_text)

('Harry had never believed he would meet a boy he hated more than Dudley, but that was before he met Draco Malfoy. Still, first-year Gryffindors only had Potions with the Slytherins, so they didn\'t have to put up with Malfoy much. Or at least, they didn\'t until they spotted a notice pinned up in the Gryffindor common room that made them all groan. Flying lessons would be starting on Thursday -- and Gryffindor and Slytherin would be learning together. \n\n "Typical," said Harry darkly. "Just what I a',
 28670)

In [17]:
len(prompt_text.split("\n\n"))

202

In [21]:
prompt_text.split("\n\n")[:2]

["Harry had never believed he would meet a boy he hated more than Dudley, but that was before he met Draco Malfoy. Still, first-year Gryffindors only had Potions with the Slytherins, so they didn't have to put up with Malfoy much. Or at least, they didn't until they spotted a notice pinned up in the Gryffindor common room that made them all groan. Flying lessons would be starting on Thursday -- and Gryffindor and Slytherin would be learning together. ",
 ' "Typical," said Harry darkly. "Just what I always wanted. To make a fool of myself on a broomstick in front of Malfoy." ']

### Restore order of word/representation

### Compute representations

In [14]:
# compute representations 
occurrences, representations = get_ordered_representations(
    ordered_strings_to_find=strings_to_find,
    prompt=prompt_text,
    tokenizer=tokenizer,
    model=model
)
print("Computed word level representations for the input sequence.")

Computed word level representations for the input sequence.


## 3. Map from word-level LLM representations to TR level representations via Lanczos resampling

In [15]:
layer_idx = -7

In [16]:
interpolated_representations = lanczosinterp2D(
    representations[layer_idx].to("cpu"),             # shape: (n_samples_input, n_dim)
    word_times,      # shape: (n_samples_input,)
    fmri_time,     # shape: (n_samples_target,)
    window=3,         # (optional) number of lobes for the window
    cutoff_mult=1.0,  # (optional) cutoff frequency multiplier
    rectify=False     # (optional) rectification
)
print("Interpolated LLM representations to match fMRI TRs via Lanczos downsampling.")

Doing lanczos interpolation with cutoff=0.500 and 3 lobes.
Interpolated LLM representations to match fMRI TRs via Lanczos downsampling.


In [17]:
interpolated_representations.shape

(1351, 2048)

## 4. Concatenate last x TRs of the LLM representation
No need to filter the first x features because we skip those anyway in the next step.

In [18]:
N_lag = 4  # number of previous TRs to concatenate
interpolated_representations = concat_past_features(torch.from_numpy(interpolated_representations), N_lag).numpy()

In [19]:
interpolated_representations.shape

(1351, 10240)

## 5. Filter out TRs at the boundary of multiple runs

In [24]:
# Delete the first 2 and last 1 elements from each experiment run
n_begin = 20
n_last = 15
mask = fmri_runs # This is an integer mask indicating which TRs belong to which run.
final_representations = delete_block_edges(interpolated_representations, 
                                           mask, 
                                           n_begin, 
                                           n_last, 
                                           axis=0) # axis 0: time-dimension is the first in our data
print(f"Deleted edges from the LLM representations to match the filtering of fMRI runs (removed first {n_begin} and last {n_last} of every run).")

Deleted edges from the LLM representations to match the filtering of fMRI runs (removed first 20 and last 15 of every run).


In [28]:
fmri_data.shape

(1211, 25263)

In [25]:
final_representations.shape

(1211, 10240)

## 6. Run the nested blocked cross-validation to find the best alpha per voxel for ridge regression.

In [29]:
# Convert data to torch tensors and move to GPU
X = torch.tensor(final_representations, dtype=torch.float32, device="cuda") # shape (n_TRs, n_features)
y = torch.tensor(fmri_data, dtype=torch.float32, device="cuda") # shape (n_TRs, n_voxels)

# Set parameters for crossvalidation
split_function = "blocked" # divide data into uniform folds
block_labels = None  # Not needed for blocked splitting - useful for experiment-wise CV folds
n_splits_outer = 4   # Four blocks for outer CV (must be larger than 1)
n_splits_inner = 3   # Three blocks for inner CV (must be larger than 1)
gap = 15 # number of TRs to skip/discard in between train and test to avoid leakage
alphas = [0.1,1,10,100,1000,10000] # default vals from https://github.com/mtoneva/brain_language_nlp/blob/master/utils/utils.py 

In [30]:
X.shape, y.shape

(torch.Size([1211, 10240]), torch.Size([1211, 25263]))

In [ ]:
device = "cuda"
print("Starting nested blocked cross-validation to find the best alpha per voxel for ridge regression")
# Obtain the best ridge regression penalties for each voxel independently (~25k alphas)
best_alphas, outer_scores = nested_blocked_cv(
    X, y,
    split_function=split_function,
    block_labels=block_labels,
    n_splits_outer=n_splits_outer,
    n_splits_inner=n_splits_inner,
    gap=gap,
    alphas=alphas,
    device=device
)
print("Best alphas found:", best_alphas)

Starting nested blocked cross-validation to find the best alpha per voxel for ridge regression


Ridge Regression models fitted:   3%|█▋                                                     | 3/96 [00:20<10:48,  6.97s/it]

torch.Size([894, 25263]) y_train_outer shape
(25263,) best alphas for fold


Ridge Regression models fitted:   6%|███▍                                                   | 6/96 [00:42<10:33,  7.04s/it]

torch.Size([894, 25263]) y_train_outer shape
(25263,) best alphas for fold


...

Ridge Regression models fitted:  58%|███████████████████████████████▌                      | 56/96 [12:24<04:43,  7.08s/it]

torch.Size([879, 25263]) y_train_outer shape
(25263,) best alphas for fold
Best alphas for fold: 25263 0 voxels with no best alpha found
(3, 25263) best alphas shape


/BRAIN/ckolling-phd/work/miniconda3/lib/python3.12/site-packages/scipy/_lib/_util.py:1226: LinAlgWarning: Ill-conditioned matrix (rcond=2.95486e-08): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
/BRAIN/ckolling-phd/work/miniconda3/lib/python3.12/site-packages/scipy/_lib/_util.py:1226: LinAlgWarning: Ill-conditioned matrix (rcond=2.95205e-08): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)
Ridge Regression models fitted:  59%|████████████████████████████████                      | 57/96 [14:47<31:09, 47.93s/it]

Outer scores for this fold: [-0.06908432 -0.02936458 -0.08971171 ... -0.10627444 -0.12139738
 -0.02587006]


Ridge Regression models fitted:  61%|█████████████████████████████████▏                    | 59/96 [15:07<09:29, 15.39s/it]


In [ ]:
block_labels = [[0 for _ in range(X.shape[0])]] # everything comes from the same story so we assume identical block_labels.